# Tests de vitesse d'exécution

In [1]:
import sqlite3
from chess_functions import *
from stats import *
import time

In [2]:
con_optimize = sqlite3.connect(":memory:")

In [3]:
con_optimize = sqlite3.connect(":memory:")
con_optimize.executescript("""
create table player (id integer not NULL primary key autoincrement, name text not NULL); 
create table tournois (id integer not NULL primary key autoincrement, name text, date text, rondes integer, cadence string, type string);
create table match (id integer primary key autoincrement, tournoi integer, black_player integer, elo_black string, white_player integer, elo_white string, winner integer);""")

In [4]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def american(number):
    
    
    
    '''
    Retourne True si la grille est de type américaine et False si elle est de type berger.

            Paramètres:
                    number (int): Un entier représentant le numéro du tournoi dans le site

            Retourne:
                    (bool): True si grille américaine et False si griller berger
    '''
    
    
    
    
    link = f"http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/{number}/{number}&Action=Ga"
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    
    for x in soup.find_all('span'):
        if x.string == "Désolé, le fichier n'existe pas...":
            print("berger")
            return False
        else :
            print("américain")
            return True 
        

def infos_tournoi(number): 
    
    
    
    '''
    Retourne le nom, la date, le nombre de rondes et la cadence d'un tournoi.

            Paramètres:
                    number (int): Un entier représentant le numéro du tournoi dans le site

            Retourne:
                    infos (dict): Dictionnaire avec comme clés la date, le nom, le nombre de rondes et la cadence du tournoi
    '''
    
    
    
    infos = dict()
    link_infos = f"http://echecs.asso.fr/FicheTournoi.aspx?Ref={number}"
    soup_infos = BeautifulSoup(requests.get(link_infos).text, 'html.parser')
    
    for x in soup_infos.find_all('span'):
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelDates":
            infos["date"] = x.string   
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelNom":
            infos["nom"] = x.string  
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelNbrRondes":
            infos["nb_rondes"] = x.string
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelCadence":
            infos["cadence"] = x.string 
    return infos





def recup_berger(number):
    
    '''
    Retourne les joueurs, les elos et les matchs d'une grille berger.

            Paramètres:
                    number (int): Un entier représentant le numéro du tournoi dans le site
            
            Retourne:
                    joueur (list): liste des joueurs du tournoi
                    elo(dict) : dictionnaire avec les noms des joueurs comme clés et les elos des joueurs comme valeurs
                    dictionnaire (dict) : dictionnaire avec les noms des joueurs comme clés et comme valeur un dictionnaire avec les scores des matchs pour chaque adversaire
                        
    '''
    
    link = f"http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/{number}/{number}&Action=Berger"
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    
    #récupération des joueurs
    j=[]
    for x in soup.find_all('td'):
        if x.has_attr("class") and x['class'][0] == "papi_border_l" :
            j.append(x.string.lower())
    joueur=j[1:]
    
    #trouver taille du tableau:a
    for k,x in enumerate(soup.find_all('tr')):
        if x.has_attr("class") and x['class'][0] == "papi_liste_c":
            if k==1:
                a=0
                for y in x.find_all('td'):
                    a+=1
    
    
    #récupération des elo
    e=[]
    i=0
    for x in soup.find_all('tr'):
        if x.has_attr("class") and x['class'][0] == "papi_liste_c":
            for k,y in enumerate(x.find_all('td')):
                if (k-3)%a==0 and k!=1 and i<len(joueur)+1:
                    z=str(y.string).split("\xa0")
                    if len(z)==2:
                        elo = z[0]+z[1]
                    else :
                        elo = z[0]
                    e.append(elo)
                    i+=1
    elo=dict()
    for i in range(len(joueur)):
        elo[joueur[i]]=e[1:][i]
    
    
    #récupération des scores relatifs sous forme de dictionnaire
    dic=dict()
    for i in range(len(joueur)):
        p=[]
        for x in soup.find_all('tr'):
            if x.has_attr("class") and x['class'][0] == "papi_liste_c":
                for k,y in enumerate(x.find_all('td')):
                    if (k-4-i)%a==0:
                        p.append(y.string)
        perf_colonne=p[1:len(joueur)+1]
        dic[joueur[i]]=perf_colonne
        #avant on a récupéré tous les modulos 15, ie on a les scores en colonne, on doit les présenter en ligne   
    
    dictionnaire=dict()
    for i in range(len(joueur)):
        dico_joueur=dictionnaire[joueur[i]]= dict()
        for j in range(len(joueur)):
            if j!=i:
                dico_joueur[joueur[j]]=dic[joueur[j]][i]
    
    return(joueur,elo,dictionnaire)       



def players_ga(number):
    
    '''
    Retourne les joueurs d'une grille berger.

            Paramètres:
                    number (int): Un entier représentant le numéro du tournoi dans le site
            
            Retourne:
                    res (list) : La liste des joueurs du tournoi
                    ind(int) : 1 si la présentation de la page est nouvelle et 2 si elle est ancienne
    '''
    
    
    indicateur = 0
    res = []
    link = f"http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/{number}/{number}&Action=Ga"
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    for x in soup.find_all('div'):
        if x.has_attr("class") and x['class'][0] == "papi_joueur_box":
            res.append(x.b.string)
    indicateur = 1
        
    if res == []:
        for k,x in enumerate(soup.find_all('tr')):
            if x.has_attr("class") and x['class'][0] == "papi_small_t":
                a=0
                for y in x.find_all('td'):
                    a+=1
        if a==18:
            col='rien'
            for (k,x) in enumerate(soup.find_all('td')) :
                if x.has_attr("class") and x["class"][0]=="papi_l" and k==19 :
                    col = x.string
                    
            if col=='Cu.' or col=="Bu." or col == "Perf" or col =="NV" or col=='Ko.' or col=='Me.' or col=='Tr.':
                for (k,x) in enumerate(soup.find_all('td')) :
                    if x.has_attr("class") and x["class"][0]=="papi_l" and (k-23)%a==0:
                        res.append(x.string)
                        
            else:
                for (k,x) in enumerate(soup.find_all('td')) :
                    if x.has_attr("class") and x["class"][0]=="papi_l" :
                        res.append(x.string)
                    
        
        elif a==20:
            for (k,x) in enumerate(soup.find_all('td')) :
                if x.has_attr("class") and x["class"][0]=="papi_l" :
                    indice = k
                
                    break
            for (k,x) in enumerate(soup.find_all('td')) :
                if x.has_attr("class") and x["class"][0]=="papi_l" and (k-(a+indice))%a==0 :
                    res.append(x.string)
               
    
        else :
            for (k,x) in enumerate(soup.find_all('td')) :
                if x.has_attr("class") and x["class"][0]=="papi_l" and (k-(5+a))%a==0:
                    res.append(x.string)
              
            if res==[]:
                for (k,x) in enumerate(soup.find_all('td')) :
                    if x.has_attr("class") and x["class"][0]=="papi_l" :
                        res.append(x.string)
                 
        res = res[1:]
        indicateur = 2
                     
    return (res, indicateur)


def match_ga(number):
    
    '''
    Retourne les matchs d'une grille américaine (adversaires, elos, gagnant).

            Paramètres:
                    number (int): Un entier représentant le numéro du tournoi dans 
                    le site
            
            Retourne:
                    res (list) : liste dont chaque élément est une liste contenant 
                    les deux joueurs, leurs elos, la couleur jouée par le premier, 
                    et 1 si le premier a gagné, 1/2 si égalité et 0 s'il a perdu. 
    '''
    
    
    symboles = ['+', '-', "=", ">", "<"]
    players, ind = players_ga(number)
    link = f"http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/{number}/{number}&Action=Ga"
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    
    res = []

    if ind== 1:
        adversaires = []
        elos = []
        couleurs = []
        résultats = []
        for x in soup.find_all('div'):
            if x.has_attr("class") and x.has_attr("align") and x['align'] == "center":
                joueur = []
                elo = []
                couleur = []
                résultat = []
                for k,y in enumerate(x.find_all('td')):
                    if (k-16)%13==0:
                        joueur.append(y.string)
                    if (k-17)%13==0:
                        z=str(y.string).split("\xa0")
                        elo.append(z)
                    if (k-12)%13==0:
                        couleur.append(y.string)
                    if k%13==0:
                        résultat.append(y.string)
                adversaires.append(joueur) 
                elos.append(elo)
                couleurs.append(couleur)
                résultats.append(résultat)
    if ind == 2:
        adversaires = []
        elos = []
        couleurs = []
        résultats = []
        for j,x in enumerate(soup.find_all('tr')):
            if j>2 and (j-3)<len(players):
                joueur = [players[j-3]]
                elo = []
                couleur = []
                résultat = []
                for k,y in enumerate(x.find_all('td')):
                    if y.has_attr("class") and (y['class'][0] == "papi_r" or y['class'][0] == "papi_c"):
                        if k==3:
                            z=str(y.string).split("\xa0")
                            elo.append(z)
                        if y.string!=None and y.string[0] in symboles and len(y.string)>4:
                            truc = y.string[:-1].split('\xa0')
                            if len(truc)>=2:
                                joueur.append(players[int(truc[-1])-1])
                                couleur.append(y.string[-1])
                                résultat.append(truc[0])
                            else : 
                                joueur.append(players[int(truc[0][1:])-1])
                                couleur.append(y.string[-1])
                                résultat.append(truc[0][0])
                        elif y.string!=None and y.string[0] in symboles and y.string[-1] in ['B', 'N'] and len(y.string)<=4:
                            
                            joueur.append(players[int(y.string[1:-1])-1])
                            couleur.append(y.string[-1])
                            résultat.append(y.string[0])
                adversaires.append(joueur) 
                elos.append(elo)
                couleurs.append(couleur)
                résultats.append(résultat)

    déjà_vus = set()
    if ind==1:
        for k in range (len(adversaires)):
            for i in range(1,len(adversaires[k])):
                match = []
                try:
                    match.append(adversaires[k][0])
                    déjà_vus.add(adversaires[k][0])
                    match.append(adversaires[k][i])
                    match.append(elos[k][0])
                    match.append(elos[k][i])
                    match.append(couleurs[k][i-1])
                    match.append(résultats[k][i])
                    if adversaires[k][i] not in déjà_vus:
                        res.append(match)
                except IndexError:
                    pass
    if ind ==2:
        for k in range (len(adversaires)):
            for i in range(1,len(adversaires[k])):
                match = []
                match.append(adversaires[k][0])
                déjà_vus.add(adversaires[k][0])
                match.append(adversaires[k][i])
                match.append(elos[k][0])
                match.append(elos[players.index(adversaires[k][i])][0])
                match.append(couleurs[k][i-1])
                match.append(résultats[k][i-1])
                if adversaires[k][i] not in déjà_vus:
                    res.append(match)
    return res


def database_old(number_min, number_max, con):
    
    '''
    Remplit la base de donnée appelée par con par les tournois numérotés de number_min
    à number_max.

            Paramètres:
                    number_min (int): Un entier représentant le numéro du premier tournoi à ajouter
                    number_max (int): Un entier représentant le numéro du dernier tournoi à ajouter
                    con (sqlite3.Connection): Une connexion à une base de donnée
                    
                    
            Retourne:
                    None
    '''
    
    
    number = number_min
    while number<number_max:
        number+=1
        print(number)
        try:
            if not stop_boucle(infos_tournoi(number)['date']):
                if american(number):
                    try:
                        insere_ga(number, con)
                    except UnboundLocalError as e:
                        print(e)
                    except IndexError as a:
                        print(a)
                else :
                    insertion_berger(number, con)
        except KeyError as e:
            print(e)         

On mesure le temps pour insérer 100 tournois dans la base de données avec le module 'html.parser' de BeautifulSoup

In [5]:
t0 = time.time()
database_old(36500, 36600, con_optimize)
tf = ( time.time() - t0 )
print (tf)

36501
américain
36502
américain
36503
américain
36504
américain
36505
américain
36506
américain
36507
berger
36508
berger
36509
américain
36510
américain
36511
américain
36512
américain
36513
américain
36514
berger
36515
berger
36516
berger
36517
berger
36518
américain
36519
américain
36520
américain
36521
américain
36522
américain
36523
américain
36524
américain
36525
américain
36526
américain
36527
berger
36528
berger
36529
berger
36530
américain
36531
américain
36532
américain
36533
berger
36534
américain
36535
américain
36536
américain
36537
américain
36538
américain
36539
américain
36540
américain
36541
berger
36542
américain
36543
américain
36544
américain
36545
berger
36546
berger
36547
américain
36548
américain
36549
américain
36550
berger
36551
américain
36552
américain
36553
américain
36554
américain
36555
berger
36556
berger
36557
américain
36558
berger
36559
américain
36560
berger
36561
américain
36562
américain
36563
américain
36564
américain
36565
américain
36566
américai

3mn42s en moyenne pour 100 matchs avec le module 'html.parser'

On mesure le temps pour insérer 100 tournois dans la base de données avec le module 'lxml' de BeautifulSoup

In [4]:
t0 = time.time()
database(20500, 20600, con_optimize)
tf = ( time.time() - t0 )
print (tf)

20501
berger
20502
américain
20503
berger
20504
berger
20505
berger
20506
berger
20507
berger
20508
berger
20509
berger
20510
berger
20511
berger
20512
berger
20513
berger
20514
berger
20515
berger
20516
berger
20517
berger
20518
berger
20519
berger
20520
berger
20521
berger
20522
berger
20523
berger
20524
berger
20525
berger
20526
berger
20527
berger
20528
berger
20529
berger
20530
berger
20531
berger
20532
berger
20533
berger
20534
américain
20535
berger
20536
berger
20537
berger
20538
berger
20539
berger
20540
berger
20541
berger
20542
berger
20543
berger
20544
berger
20545
berger
20546
berger
20547
berger
20548
berger
20549
berger
20550
berger
20551
berger
20552
berger
20553
berger
20554
berger
20555
berger
20556
berger
20557
berger
20558
berger
20559
berger
20560
berger
20561
berger
20562
berger
20563
berger
20564
berger
20565
berger
20566
berger
20567
berger
20568
berger
20569
berger
20570
berger
20571
berger
20572
berger
20573
berger
20574
berger
20575
berger
20576
berger
20577


1mn14 pour 100 matchs en moyenne avec "lxml".